### Etivity 5- PCA

#### Student Name:   Mark Murnane

#### Student ID:     18195326

- The 'as' keyword allows you to invoke functionality from the module using an alias for the module name. For example: np.mean() instead of numpy.mean()
- The from keyword allows you to only import the functionality of interest, for example above we import only the PCA class from the sklearn.decomposition module

In [1]:
import numpy as np
import random as rand
import matplotlib.pyplot as plt
from numpy.linalg import eig
from sklearn.decomposition import PCA

As per E-tivity instructions: Use of the matrix class is discouraged, but to allow us to simplify the code slightly, we will use it this week. Its first use will be to store the data that you will perform the PCA transform on. Note that you will likely obtain a higher score if your final version does not use the matrix class.

In [2]:
a_x = 0.05
a_y= 10

In [3]:
data =  np.array([[n*(1+a_x*(rand.random()-0.5)),4*n+ a_y*(rand.random()-0.5)] for n in range(20)])
data

array([[ 0.        ,  3.02643882],
       [ 1.00302468,  3.77581571],
       [ 2.01834168,  4.77098768],
       [ 2.98684566, 14.1154116 ],
       [ 3.96436336, 15.80340231],
       [ 5.0356314 , 15.24896614],
       [ 6.04005009, 26.80550406],
       [ 6.94104483, 23.74771991],
       [ 7.92010654, 35.58240131],
       [ 9.2201525 , 33.76838478],
       [10.2426964 , 37.63014143],
       [10.79983959, 45.95119884],
       [11.71261515, 48.51996017],
       [13.25754503, 51.16668769],
       [13.79980117, 51.105388  ],
       [15.14412871, 63.66125497],
       [16.1666178 , 59.24647693],
       [16.89223507, 67.10744822],
       [17.80992819, 67.48190716],
       [18.81448752, 74.333798  ]])

The numpy shape property is very useful to get an insight in the dimensions of your data, for example to check whether the features (in this case 2) or the samples (in this case 20) are in the rows or the columns. The notation used here (with columns containing the features and rows containing separate examples) is the standard for Scikitlearn and many other machine learning algorithms.


In [4]:
data.shape

(20, 2)

### Scikit

The `fit()` values from Scikitlearn are:

In [15]:
pca = PCA(n_components=2)
pca.fit(data)
print(pca.components_)

[[ 0.24596569  0.96927853]
 [-0.96927853  0.24596569]]


### My Implementation

I changed the definition of the data from an `np.matrix` to an `np.array`.

In [13]:
class myPCA(object):
    def __init__(self, n_components):
        self.num_components = n_components
        
    def fit (self, A):
        # Here we'll calculate the Eigen vectors and Eigen values
        
        # First of all, calculate the mean for each of the columns
        self.mean = np.array([np.mean(A[:,column]) for column in range(A.shape[1])])
        
        # Then calculate the centred matrix, which deducts the mean from each value
        self.centered = A - self.mean
        
        # Calculate the covariant matrix for the centered matrix.
        self.covariant = np.cov(self.centered, rowvar=False)
                
        # Next, generate the Eigen values and Eigen vectors for the covariant matrix.
        self.e_values, self.e_vectors = eig(self.covariant)
        
    def transform(self):
        pass
        
myP = myPCA(2)
myP.fit(data)
print(f"The Eigen values of the data are {myP.e_values}")
print(f"The Eigen vectors of the data are\n{myP.e_vectors}")
print()

The Eigen values of the data are [5.48592104e-01 5.64794103e+02]
The Eigen vectors of the data are
[[-0.96927853 -0.24596569]
 [ 0.24596569 -0.96927853]]



In the above `fit` method it was necessary to override the default behaviour of NumPy's `cov()` function.  It assumes that each row in the data is a variable, with the column values of that row the data set for that variable.  Our data set is organised with each column representing a variable, and the rows containg the data.  One option was to _transpose_ the centered matrix; the other was to tell `np.cov()` to use columns as data sets.

#### Output comparison

The Eigen vectors produced by my calculation have components with the same magnitude as those produced by Scikitlean, but not the same sign or position.  Needs further investigation to understand.